# Feature Engineering

## 1. Setup and Data Loading

In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
import warnings
warnings.filterwarnings('ignore')

# Set plotting styles
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("deep")
plt.rcParams['figure.figsize'] = [12, 6]

# Define paths
DATA_PATH = Path("../data")
RAW_DATA_PATH = DATA_PATH / "raw"
PROCESSED_DATA_PATH = DATA_PATH / "processed"

# Create processed data directory if it doesn't exist
if not os.path.exists(PROCESSED_DATA_PATH):
    os.makedirs(PROCESSED_DATA_PATH)

# Load the raw data
def load_data():
    """Load all raw data sources"""
    print("Loading data files...")
    
    economic_indicators = pd.read_parquet(RAW_DATA_PATH / "economic_indicators.parquet")
    fama_french_factors = pd.read_parquet(RAW_DATA_PATH / "fama_french_factors.parquet")
    stock_prices = pd.read_parquet(RAW_DATA_PATH / "stock_prices.parquet")
    world_bank_data = pd.read_parquet(RAW_DATA_PATH / "world_bank_data.parquet")
    
    print(f"Economic Indicators: {economic_indicators.shape}")
    print(f"Fama-French Factors: {fama_french_factors.shape}")
    print(f"Stock Prices: {stock_prices.shape}")
    print(f"World Bank Data: {world_bank_data.shape}")
    
    return economic_indicators, fama_french_factors, stock_prices, world_bank_data

economic_indicators, fama_french_factors, stock_prices, world_bank_data = load_data()

Loading data files...
Economic Indicators: (4005, 5)
Fama-French Factors: (180, 4)
Stock Prices: (3812, 48)
World Bank Data: (70, 6)


## 2. Exploratory Data Analysis

### 2.1 Economic Indicators Exploration

In [3]:
print("Economic Indicators - First 5 rows:")
display(economic_indicators.head())
print("\nEconomic Indicators - Information:")
display(economic_indicators.info())
print("\nMissing values by column:")
display(economic_indicators.isna().sum())

# Brief statistical summary
display(economic_indicators.describe())

# Check frequency of data
def check_data_frequency(df, date_column='date'):
    """Analyze the frequency of time series data"""
    if date_column in df.columns:
        df = df.set_index(date_column) if df.index.name != date_column else df
    
    # Calculate differences between consecutive dates
    date_diffs = df.index.to_series().diff().value_counts().sort_index()
    print("Date differences (frequency):")
    display(date_diffs)
    
    return date_diffs

check_data_frequency(economic_indicators)

Economic Indicators - First 5 rows:


,GDP,UNRATE,CPIAUCSL,FEDFUNDS,T10Y2Y
2010-01-01,14764.61,9.8,217.488,0.11,NaN
2010-01-04,NaN,NaN,NaN,NaN,2.76
2010-01-05,NaN,NaN,NaN,NaN,2.76
2010-01-06,NaN,NaN,NaN,NaN,2.84
2010-01-07,NaN,NaN,NaN,NaN,2.82



Economic Indicators - Information:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4005 entries, 2010-01-01 to 2025-02-27
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   GDP       60 non-null     float64
 1   UNRATE    181 non-null    float64
 2   CPIAUCSL  181 non-null    float64
 3   FEDFUNDS  181 non-null    float64
 4   T10Y2Y    3791 non-null   float64
dtypes: float64(5)
memory usage: 187.7 KB


None


Missing values by column:


GDP         3945
UNRATE      3824
CPIAUCSL    3824
FEDFUNDS    3824
T10Y2Y       214
dtype: int64

,GDP,UNRATE,CPIAUCSL,FEDFUNDS,T10Y2Y
count,60.000000,181.000000,181.000000,181.000000,3791.000000
mean,20549.782617,5.788398,254.491083,1.246409,1.012933
std,4310.087947,2.234609,28.788096,1.728006,0.970092
min,14764.610000,3.400000,217.199000,0.050000,-1.080000
25%,17132.473750,3.900000,233.669000,0.090000,0.240000
50%,19565.619000,5.000000,244.243000,0.190000,1.010000
75%,22834.810000,7.500000,266.625000,1.830000,1.710000
max,29719.647000,14.800000,319.086000,5.330000,2.910000


Date differences (frequency):


1 days    3213
2 days      50
3 days     741
Name: count, dtype: int64

1 days    3213
2 days      50
3 days     741
Name: count, dtype: int64

### 2.2 Fama-French Factors Exploration

In [4]:
print("Fama-French Factors - First 5 rows:")
display(fama_french_factors.head())
print("\nFama-French Factors - Information:")
display(fama_french_factors.info())
check_data_frequency(fama_french_factors)

Fama-French Factors - First 5 rows:


,Mkt-RF,SMB,HML,RF
Date,,,,
2010-01,-0.0336,0.0040,0.0043,0.0000
2010-02,0.0340,0.0119,0.0322,0.0000
2010-03,0.0631,0.0148,0.0221,0.0001
2010-04,0.0200,0.0487,0.0289,0.0001
2010-05,-0.0789,0.0009,-0.0244,0.0001



Fama-French Factors - Information:
<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 180 entries, 2010-01 to 2024-12
Freq: M
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Mkt-RF  180 non-null    float64
 1   SMB     180 non-null    float64
 2   HML     180 non-null    float64
 3   RF      180 non-null    float64
dtypes: float64(4)
memory usage: 7.0 KB


None

Date differences (frequency):


Date
<MonthEnd>    179
Name: count, dtype: int64

Date
<MonthEnd>    179
Name: count, dtype: int64

### 2.3 Stock Prices Exploration

In [5]:
print("Stock Prices - First 5 rows:")
display(stock_prices.head())
print("\nStock Prices - Information:")
display(stock_prices.info())
print("\nStock Prices - Column names and types:")
for col in stock_prices.columns:
    print(f"{col} - {type(col)}")
check_data_frequency(stock_prices)

# Determine if we have a ticker/symbol column
def find_ticker_column(df):
    ticker_col = None
    # List of keywords to look for in column names
    ticker_keywords = ['ticker', 'symbol', 'stock', 'asset', 'security']
    
    # Check for exact matches first
    if 'ticker' in df.columns:
        ticker_col = 'ticker'
    elif 'symbol' in df.columns:
        ticker_col = 'symbol'
    else:
        # Look for columns containing keywords, handling both string and tuple column names
        for col in df.columns:
            # For string column names
            if isinstance(col, str) and any(keyword in col.lower() for keyword in ticker_keywords):
                ticker_col = col
                break
            # For tuple column names, check each element in the tuple
            elif isinstance(col, tuple):
                for item in col:
                    if isinstance(item, str) and any(keyword in item.lower() for keyword in ticker_keywords):
                        ticker_col = col
                        break
                if ticker_col:  # Break out of outer loop if we found a match
                    break
    
    if ticker_col:
        print(f"Using '{ticker_col}' as the ticker/symbol identifier column.")
    else:
        print("No ticker/symbol column found. This might be data for a single stock or an index.")
    
    return ticker_col

# Find ticker column
ticker_col = find_ticker_column(stock_prices)

# Print stock identifiers if available
if ticker_col:
    print(f"Number of unique stocks: {stock_prices[ticker_col].nunique()}")
    print(f"Sample {ticker_col} values:")
    display(stock_prices[ticker_col].unique()[:10])
else:
    print("Processing as single time series data.")


Stock Prices - First 5 rows:


Ticker             GLD                                                \
Price             Open        High         Low       Close    Volume   
Date                                                                   
2010-01-04  109.820000  110.139999  109.309998  109.800003  16224100   
2010-01-05  109.879997  110.389999  109.260002  109.699997  14213100   
2010-01-06  110.709999  111.769997  110.410004  111.510002  24981900   
2010-01-07  111.070000  111.290001  110.620003  110.820000  13609800   
2010-01-08  111.519997  111.580002  110.260002  111.370003  15894600   

Ticker                                                 AGG             ...  \
Price      Dividends Stock Splits Capital Gains       Open       High  ...   
Date                                                                   ...   
2010-01-04       0.0          0.0           0.0  68.665953  68.759046  ...   
2010-01-05       0.0          0.0           0.0  68.898712  69.031694  ...   
2010-01-06       0.0          0.0           0.0  69.031678  69.031678  ...   
2010-01-07       0.0          0.0           0.0  68.925351  68.958594  ...   
2010-01-08       0.0          0.0           0.0  69.065001  69.065001  ...   

Ticker              VEA                      SPY                        \
Price      Stock Splits Capital Gains       Open       High        Low   
Date                                                                     
2010-01-04          0.0           0.0  85.297736  86.071994  84.644927   
2010-01-05          0.0           0.0  85.973302  86.292114  85.662077   
2010-01-06          0.0           0.0  86.170669  86.527437  86.102354   
2010-01-07          0.0           0.0  86.155501  86.785539  85.912596   
2010-01-08          0.0           0.0  86.451523  87.005653  86.276938   

Ticker                                                                 
Price           Close     Volume Dividends Stock Splits Capital Gains  
Date                                                                   
2010-01-04  86.026451  118944600       0.0          0.0           0.0  
2010-01-05  86.254158  111579900       0.0          0.0           0.0  
2010-01-06  86.314896  116074400       0.0          0.0           0.0  
2010-01-07  86.679268  131091100       0.0          0.0           0.0  
2010-01-08  86.967697  126402800       0.0          0.0           0.0  

[5 rows x 48 columns]


Stock Prices - Information:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3812 entries, 2010-01-04 to 2025-02-27
Data columns (total 48 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   (GLD, Open)           3812 non-null   float64
 1   (GLD, High)           3812 non-null   float64
 2   (GLD, Low)            3812 non-null   float64
 3   (GLD, Close)          3812 non-null   float64
 4   (GLD, Volume)         3812 non-null   int64  
 5   (GLD, Dividends)      3812 non-null   float64
 6   (GLD, Stock Splits)   3812 non-null   float64
 7   (GLD, Capital Gains)  3812 non-null   float64
 8   (AGG, Open)           3812 non-null   float64
 9   (AGG, High)           3812 non-null   float64
 10  (AGG, Low)            3812 non-null   float64
 11  (AGG, Close)          3812 non-null   float64
 12  (AGG, Volume)         3812 non-null   int64  
 13  (AGG, Dividends)      3812 non-null   float64
 14  (AGG, Stock Splits)   381

None


Stock Prices - Column names and types:
('GLD', 'Open') - <class 'tuple'>
('GLD', 'High') - <class 'tuple'>
('GLD', 'Low') - <class 'tuple'>
('GLD', 'Close') - <class 'tuple'>
('GLD', 'Volume') - <class 'tuple'>
('GLD', 'Dividends') - <class 'tuple'>
('GLD', 'Stock Splits') - <class 'tuple'>
('GLD', 'Capital Gains') - <class 'tuple'>
('AGG', 'Open') - <class 'tuple'>
('AGG', 'High') - <class 'tuple'>
('AGG', 'Low') - <class 'tuple'>
('AGG', 'Close') - <class 'tuple'>
('AGG', 'Volume') - <class 'tuple'>
('AGG', 'Dividends') - <class 'tuple'>
('AGG', 'Stock Splits') - <class 'tuple'>
('AGG', 'Capital Gains') - <class 'tuple'>
('VNQ', 'Open') - <class 'tuple'>
('VNQ', 'High') - <class 'tuple'>
('VNQ', 'Low') - <class 'tuple'>
('VNQ', 'Close') - <class 'tuple'>
('VNQ', 'Volume') - <class 'tuple'>
('VNQ', 'Dividends') - <class 'tuple'>
('VNQ', 'Stock Splits') - <class 'tuple'>
('VNQ', 'Capital Gains') - <class 'tuple'>
('VWO', 'Open') - <class 'tuple'>
('VWO', 'High') - <class 'tuple'>
('VW

Date
1 days    2984
2 days      37
3 days     686
4 days     103
5 days       1
Name: count, dtype: int64

Using '('GLD', 'Stock Splits')' as the ticker/symbol identifier column.
Number of unique stocks: 1
Sample ('GLD', 'Stock Splits') values:


array([0.])

### 2.4 World Bank Data Exploration

In [6]:
print("World Bank Data - First 5 rows:")
display(world_bank_data.head())
print("\nWorld Bank Data - Information:")
display(world_bank_data.info())

# World Bank data is often formatted differently - let's examine the structure
print("\nWorld Bank Data - Column names:")
print(world_bank_data.columns.tolist())

# Check if there's a date/year column
date_col = None
for col in world_bank_data.columns:
    col_name = col if isinstance(col, str) else str(col)
    if any(keyword in col_name.lower() for keyword in ['date', 'year', 'time', 'period']):
        date_col = col
        print(f"Potential date column found: {col}")
        print(f"Sample values: {world_bank_data[col].iloc[:5].tolist()}")
        break

if date_col:
    try:
        # Try to convert to datetime for frequency analysis
        world_bank_data_copy = world_bank_data.copy()
        world_bank_data_copy[date_col] = pd.to_datetime(world_bank_data_copy[date_col])
        check_data_frequency(world_bank_data_copy, date_column=date_col)
    except Exception as e:
        print(f"Could not convert World Bank date column to datetime: {e}")
        # If conversion fails, try checking if it's annual data with year values
        if world_bank_data[date_col].dtype in ['int64', 'float64'] or world_bank_data[date_col].str.isnumeric().all():
            print("Data appears to be annual with year values.")
else:
    print("No obvious date column found. Examining shape of data instead.")
    print(f"Number of rows: {world_bank_data.shape[0]}")
    print(f"Number of columns: {world_bank_data.shape[1]}")
    
    # Check if columns might be years (common in World Bank data)
    numeric_cols = [col for col in world_bank_data.columns if isinstance(col, (int, float)) or 
                   (isinstance(col, str) and col.isdigit())]
    if numeric_cols:
        print("Some column names appear to be years:")
        print(sorted(numeric_cols)[:10])  # Show first 10 years
        print("This suggests wide-format data with years as columns.")
        
        # If needed, we can reshape to long format
        print("\nExample of how to reshape to long format:")
        print("world_bank_long = world_bank_data.melt(id_vars=['country', 'indicator'], var_name='year', value_name='value')")
        
        # Try to identify ID columns (non-year columns)
        id_cols = [col for col in world_bank_data.columns if col not in numeric_cols]
        if id_cols:
            print("\nNon-year columns that could be identifiers:")
            print(id_cols)

World Bank Data - First 5 rows:


,country,date,GDP Growth,Inflation,Trade % GDP,year
0,China,2023,5.249558,0.234837,37.316771,2023
1,China,2022,2.950670,1.973576,38.351482,2022
2,China,2021,8.448469,0.981015,37.301991,2021
3,China,2020,2.238638,2.419422,34.754296,2020
4,China,2019,5.950501,2.899234,35.890096,2019



World Bank Data - Information:
<class 'pandas.core.frame.DataFrame'>
Index: 70 entries, 0 to 269
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   country      70 non-null     object 
 1   date         70 non-null     object 
 2   GDP Growth   70 non-null     float64
 3   Inflation    70 non-null     float64
 4   Trade % GDP  70 non-null     float64
 5   year         70 non-null     int64  
dtypes: float64(3), int64(1), object(2)
memory usage: 3.8+ KB


None


World Bank Data - Column names:
['country', 'date', 'GDP Growth', 'Inflation', 'Trade % GDP', 'year']
Potential date column found: date
Sample values: ['2023', '2022', '2021', '2020', '2019']
Date differences (frequency):


date
-366 days    15
-365 days    50
4748 days     4
Name: count, dtype: int64

## 3. Data Preprocessing and Alignment

### 3.1 Time Series Alignment

In [7]:
def align_time_series_data(dataframes, date_col='date', freq='M', methods=None, ticker_cols=None):
    """
    Align multiple time series dataframes to a common frequency
    
    Args:
        dataframes: List of dataframes to align
        date_col: Name of the date column
        freq: Target frequency ('D' for daily, 'W' for weekly, 'M' for monthly)
        methods: List of methods to use for resampling each dataframe
                ('mean', 'last', 'first', etc.)
        ticker_cols: List of column names that identify individual assets in each dataframe
                    (None for dataframes without asset identifiers)
    
    Returns:
        List of aligned dataframes
    """
    if methods is None:
        methods = ['last'] * len(dataframes)
    
    if ticker_cols is None:
        ticker_cols = [None] * len(dataframes)
    
    aligned_dfs = []
    
    for df, method, ticker_col in zip(dataframes, methods, ticker_cols):
        # Make sure index is datetime
        if date_col in df.columns:
            df = df.set_index(date_col)
        
        # For stock data with multiple tickers, handle differently
        if ticker_col and ticker_col in df.columns:
            # Example: Process each ticker separately and then combine
            tickers = df[ticker_col].unique()
            aligned_ticker_dfs = []
            
            for ticker in tickers:
                ticker_df = df[df[ticker_col] == ticker].copy()
                # Resample ticker data
                resampled = ticker_df.resample(freq).agg(method)
                resampled[ticker_col] = ticker
                aligned_ticker_dfs.append(resampled)
            
            aligned_df = pd.concat(aligned_ticker_dfs)
        else:
            # For single time series dataframes (economic indicators, factors)
            aligned_df = df.resample(freq).agg(method)
        
        aligned_dfs.append(aligned_df)
    
    return aligned_dfs

# Determine appropriate resampling methods for each dataset
resample_methods = [
    {'date': 'first', 'numerical_cols': 'last'},  # Economic indicators
    {'date': 'first', 'numerical_cols': 'mean'},  # Fama-French
    {'date': 'first', 'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'},  # Stock prices
    {'date': 'first', 'numerical_cols': 'last'}   # World Bank data
]

### 3.2 Handle Missing Values

In [8]:
def handle_missing_values(df, method='ffill'):
    """Fill missing values in the dataframe"""
    if method == 'ffill':
        return df.ffill()
    elif method == 'bfill':
        return df.bfill()
    elif method == 'interpolate':
        return df.interpolate(method='linear')
    else:
        return df

## 4. Feature Engineering for Stock Data

In [9]:
def calculate_return_features(df, price_col='close', periods=[1, 5, 20, 60, 120, 252]):
    """
    Calculate various return-based features
    
    Args:
        df: Stock price dataframe
        price_col: Column containing price data
        periods: List of periods for calculating returns
    
    Returns:
        Dataframe with added return features
    """
    result = df.copy()
    
    # Simple returns
    for period in periods:
        result[f'return_{period}d'] = result[price_col].pct_change(period)
    
    # Log returns (useful for mathematical operations)
    result['log_return_1d'] = np.log(result[price_col] / result[price_col].shift(1))
    
    # Cumulative returns
    result['cum_return_ytd'] = result.groupby([result.index.year, 'ticker'])[price_col].apply(
        lambda x: x / x.iloc[0] - 1)
    
    # Rolling return metrics
    result['rolling_mean_20d'] = result['return_1d'].rolling(window=20).mean()
    result['rolling_std_20d'] = result['return_1d'].rolling(window=20).std()
    
    # Risk-adjusted return (Sharpe ratio proxy - without risk-free rate)
    result['rolling_sharpe_20d'] = result['rolling_mean_20d'] / result['rolling_std_20d']
    
    return result

def calculate_volatility_features(df, return_col='return_1d', windows=[20, 60, 120]):
    """Calculate volatility-based features"""
    result = df.copy()
    
    # Historical volatility (annualized)
    for window in windows:
        result[f'volatility_{window}d'] = result[return_col].rolling(window=window).std() * np.sqrt(252)
    
    # Calculate high-low range volatility
    if 'high' in result.columns and 'low' in result.columns:
        result['daily_range'] = (result['high'] - result['low']) / result['close']
        result['rolling_range_20d'] = result['daily_range'].rolling(window=20).mean()
    
    return result

def calculate_technical_indicators(df, close_col='close', high_col='high', low_col='low', volume_col='volume'):
    """Calculate common technical indicators"""
    result = df.copy()
    
    # Moving Averages
    for window in [20, 50, 200]:
        result[f'ma_{window}d'] = result[close_col].rolling(window=window).mean()
    
    # MACD components
    result['ema_12d'] = result[close_col].ewm(span=12, adjust=False).mean()
    result['ema_26d'] = result[close_col].ewm(span=26, adjust=False).mean()
    result['macd'] = result['ema_12d'] - result['ema_26d']
    result['macd_signal'] = result['macd'].ewm(span=9, adjust=False).mean()
    result['macd_hist'] = result['macd'] - result['macd_signal']
    
    # Relative Strength Index (RSI)
    delta = result[close_col].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    
    # Avoid division by zero
    rs = gain / loss.replace(0, np.nan)
    result['rsi_14d'] = 100 - (100 / (1 + rs))
    
    # Bollinger Bands
    result['bb_middle_20d'] = result[close_col].rolling(window=20).mean()
    result['bb_std_20d'] = result[close_col].rolling(window=20).std()
    result['bb_upper_20d'] = result['bb_middle_20d'] + 2 * result['bb_std_20d']
    result['bb_lower_20d'] = result['bb_middle_20d'] - 2 * result['bb_std_20d']
    
    # Relative volume
    result['volume_ratio_20d'] = result[volume_col] / result[volume_col].rolling(window=20).mean()
    
    return result

def calculate_momentum_features(df, close_col='close', periods=[20, 60, 120, 252]):
    """Calculate momentum-based features"""
    result = df.copy()
    
    # Momentum (price change over period)
    for period in periods:
        result[f'momentum_{period}d'] = result[close_col].diff(period)
    
    # Rate of Change (percent price change over period)
    for period in periods:
        result[f'roc_{period}d'] = result[close_col].pct_change(period) * 100
    
    # Recent performance percentile (where current price ranks in its recent history)
    for period in periods:
        result[f'price_percentile_{period}d'] = result[close_col].rolling(period).apply(
            lambda x: pd.Series(x).rank(pct=True).iloc[-1])
    
    return result

def calculate_drawdown_features(df, return_col='return_1d'):
    """Calculate drawdown-related features"""
    result = df.copy()
    
    # Calculate cumulative returns
    result['cum_return'] = (1 + result[return_col]).cumprod()
    
    # Calculate running maximum
    result['running_max'] = result['cum_return'].cummax()
    
    # Calculate drawdown
    result['drawdown'] = result['cum_return'] / result['running_max'] - 1
    
    # Rolling maximum drawdown
    result['max_drawdown_60d'] = result['drawdown'].rolling(window=60).min()
    
    return result

## 5. Feature Engineering for Economic Data

In [10]:
def calculate_economic_indicators_features(df):
    """Generate features from economic indicators"""
    result = df.copy()
    
    # Calculate changes
    numeric_cols = result.select_dtypes(include=['number']).columns
    
    for col in numeric_cols:
        # Percentage change
        result[f'{col}_pct_change'] = result[col].pct_change()
        
        # Z-score (standardization)
        result[f'{col}_zscore'] = (result[col] - result[col].rolling(window=60).mean()) / result[col].rolling(window=60).std()
        
        # Moving averages
        result[f'{col}_ma_3m'] = result[col].rolling(window=3).mean()
        result[f'{col}_ma_12m'] = result[col].rolling(window=12).mean()
        
        # Trend indicators (difference between short and long moving averages)
        result[f'{col}_trend'] = result[f'{col}_ma_3m'] - result[f'{col}_ma_12m']
    
    return result

def calculate_yield_curve_features(df, columns=['yield_3m', 'yield_2y', 'yield_5y', 'yield_10y', 'yield_30y']):
    """Calculate yield curve related features"""
    result = df.copy()
    
    # Ensure we have required columns
    if all(col in df.columns for col in columns):
        # Term spreads (differences between yields of different maturities)
        result['term_spread_10y_3m'] = result['yield_10y'] - result['yield_3m']
        result['term_spread_10y_2y'] = result['yield_10y'] - result['yield_2y']
        result['term_spread_30y_10y'] = result['yield_30y'] - result['yield_10y']
        
        # Yield curve steepness (difference between longest and shortest yield)
        result['yield_curve_steepness'] = result['yield_30y'] - result['yield_3m']
        
        # Yield curve curvature
        result['yield_curve_curvature'] = (result['yield_5y'] * 2) - result['yield_2y'] - result['yield_10y']
        
        # Moving averages and trends of spreads
        result['term_spread_10y_3m_ma_12m'] = result['term_spread_10y_3m'].rolling(window=12).mean()
        result['term_spread_10y_3m_trend'] = result['term_spread_10y_3m'] - result['term_spread_10y_3m_ma_12m']
    
    return result

## 6. Market Regime Features

In [11]:
def engineer_market_regime_features(stock_data, economic_data, fama_french_data, ticker_col=None):
    """Create features for market regime classification"""
    # Make a copy to avoid modifying the original
    stock_data_copy = stock_data.copy()
    
    # Check if we need to handle multiple tickers
    if ticker_col and ticker_col in stock_data_copy.columns:
        # Create market aggregate by averaging across tickers
        aggregation_cols = {
            'return_1d': 'mean',
            'volatility_20d': 'mean',
            'drawdown': 'mean'
        }
        
        # Add volume ratio if it exists
        if 'volume_ratio_20d' in stock_data_copy.columns:
            aggregation_cols['volume_ratio_20d'] = 'mean'
        
        # Group by date and aggregate
        market_data = stock_data_copy.groupby(stock_data_copy.index).agg(aggregation_cols)
    else:
        # Single time series, just use it directly as market data
        market_data = stock_data_copy
    
    # Rename columns to indicate market level
    rename_dict = {}
    if 'return_1d' in market_data.columns:
        rename_dict['return_1d'] = 'market_return'
    if 'volatility_20d' in market_data.columns:
        rename_dict['volatility_20d'] = 'market_volatility'
    if 'volume_ratio_20d' in market_data.columns:
        rename_dict['volume_ratio_20d'] = 'market_volume_ratio'
    if 'drawdown' in market_data.columns:
        rename_dict['drawdown'] = 'market_drawdown'
    
    market_data = market_data.rename(columns=rename_dict)
    
    # Make sure we have market_return for further analysis
    if 'market_return' not in market_data.columns:
        if 'return_1d' in stock_data_copy.columns:
            market_data['market_return'] = stock_data_copy['return_1d']
        else:
            print("Warning: No return data found. Using random values for demonstration.")
            market_data['market_return'] = np.random.normal(0.0001, 0.01, size=len(market_data))
    
    # Make sure we have market_volatility for further analysis
    if 'market_volatility' not in market_data.columns:
        if 'volatility_20d' in stock_data_copy.columns:
            market_data['market_volatility'] = stock_data_copy['volatility_20d']
        else:
            print("Warning: No volatility data found. Calculating from returns.")
            market_data['market_volatility'] = market_data['market_return'].rolling(window=20).std() * np.sqrt(252)
    
    # Add rolling statistics
    market_data['rolling_return_20d'] = market_data['market_return'].rolling(window=20).mean() * 20  # Approx monthly return
    market_data['rolling_return_60d'] = market_data['market_return'].rolling(window=60).mean() * 60  # Approx quarterly return
    
    # Volatility regime features (using ffill to handle NaNs)
    market_data['market_volatility_filled'] = market_data['market_volatility'].fillna(method='ffill')
    
    # Check if we have enough data for quantiles
    if len(market_data) >= 4:  # Need at least 4 points for 4 quantiles
        try:
            market_data['volatility_regime'] = pd.qcut(
                market_data['market_volatility_filled'], 
                q=4, 
                labels=['low', 'medium-low', 'medium-high', 'high']
            )
        except ValueError as e:
            print(f"Warning: Could not create volatility regimes: {e}")
            # Create a simplified version with just high/low split
            median_vol = market_data['market_volatility_filled'].median()
            market_data['volatility_regime'] = np.where(
                market_data['market_volatility_filled'] > median_vol, 
                'high', 
                'low'
            )
    else:
        print("Warning: Not enough data for volatility regime classification.")
        market_data['volatility_regime'] = 'undefined'
    
    # Trend regime features (simplified version)
    market_data['market_ma_20d'] = market_data['market_return'].rolling(window=20).mean()
    
    # Use a smaller window if we don't have enough data
    ma_long_window = min(100, max(30, len(market_data) // 3))
    market_data['market_ma_long'] = market_data['market_return'].rolling(window=ma_long_window).mean()
    market_data['trend_indicator'] = np.where(
        market_data['market_ma_20d'] > market_data['market_ma_long'], 
        1, 
        -1
    )
    
    # Join with economic indicators if available
    if economic_data is not None and not economic_data.empty:
        regime_features = market_data.join(economic_data, how='left')
    else:
        regime_features = market_data
    
    # Add Fama-French factors if available
    if fama_french_data is not None and not fama_french_data.empty:
        # Check if the required columns exist
        ff_cols = [col for col in ['SMB', 'HML'] if col in fama_french_data.columns]
        
        if ff_cols:
            regime_features = regime_features.join(fama_french_data[ff_cols], how='left')
            
            # Calculate rolling factor performance
            for factor in ff_cols:
                regime_features[f'{factor}_60d'] = fama_french_data[factor].rolling(window=60).mean() * 60
                regime_features[f'{factor}_regime'] = np.sign(regime_features[f'{factor}_60d'])
    
    # Basic market regime classification
    if all(col in regime_features.columns for col in ['market_volatility', 'market_return']):
        # Use 75th percentile for volatility threshold
        vol_threshold = regime_features['market_volatility'].quantile(0.75)
        
        conditions = [
            (regime_features['market_volatility'] > vol_threshold) & 
            (regime_features['market_return'] < 0),
            
            (regime_features['market_volatility'] <= vol_threshold) & 
            (regime_features['market_return'] < 0),
            
            (regime_features['market_volatility'] > vol_threshold) & 
            (regime_features['market_return'] >= 0),
            
            (regime_features['market_volatility'] <= vol_threshold) & 
            (regime_features['market_return'] >= 0)
        ]
        
        choices = ['bear_volatile', 'bear_stable', 'bull_volatile', 'bull_stable']
        regime_features['market_regime_basic'] = np.select(conditions, choices, default='undefined')
    
    return regime_features

## 7. Apply Feature Engineering

In [12]:
# Check if required columns exist and identify column names
def identify_price_columns(df):
    """Identify price and volume columns in the dataframe"""
    col_mapping = {}
    
    # Helper function to check if a column name (string or tuple) contains a keyword
    def column_contains_keyword(col, keyword):
        if isinstance(col, str):
            return keyword.lower() in col.lower()
        elif isinstance(col, tuple):
            return any(isinstance(item, str) and keyword.lower() in item.lower() for item in col)
        return False
    
    # Look for close price column
    close_candidates = [
        col for col in df.columns if column_contains_keyword(col, 'close') or 
                                    column_contains_keyword(col, 'price') or 
                                    column_contains_keyword(col, 'adj')
    ]
    
    if close_candidates:
        col_mapping['close'] = close_candidates[0]
        print(f"Using '{close_candidates[0]}' as the close price column.")
    else:
        # If no obvious price column, use the first numeric column
        numeric_cols = df.select_dtypes(include=['number']).columns
        if len(numeric_cols) > 0:
            col_mapping['close'] = numeric_cols[0]
            print(f"Using '{numeric_cols[0]}' as the price column (fallback).")
        else:
            print("Warning: No suitable price column found.")
            col_mapping['close'] = df.columns[0]  # Last resort fallback
    
    # Look for high price column
    high_candidates = [col for col in df.columns if column_contains_keyword(col, 'high')]
    if high_candidates:
        col_mapping['high'] = high_candidates[0]
    else:
        col_mapping['high'] = col_mapping.get('close')  # Fallback to close price
    
    # Look for low price column
    low_candidates = [col for col in df.columns if column_contains_keyword(col, 'low')]
    if low_candidates:
        col_mapping['low'] = low_candidates[0]
    else:
        col_mapping['low'] = col_mapping.get('close')  # Fallback to close price
    
    # Look for volume column
    volume_candidates = [
        col for col in df.columns if column_contains_keyword(col, 'volume') or 
                                    column_contains_keyword(col, 'vol')
    ]
    
    if volume_candidates:
        col_mapping['volume'] = volume_candidates[0]
    else:
        # If no volume column, create a dummy constant volume
        print("Warning: No volume column found. Creating dummy volume.")
        df['dummy_volume'] = 1
        col_mapping['volume'] = 'dummy_volume'
    
    return col_mapping

# Process stock data
print("Engineering features for stock data...")
stock_features = stock_prices.copy()

# Find ticker column if it exists
ticker_col = find_ticker_column(stock_features)

# Identify price columns
price_cols = identify_price_columns(stock_features)
print(f"Identified columns: {price_cols}")

# Apply feature engineering with identified columns

# Define a modified version of calculate_return_features that supports ticker_col and tuple columns
def calculate_return_features_with_ticker(df, price_col='close', periods=[1, 5, 20, 60, 120, 252], ticker_col=None):
    """
    Calculate various return-based features with support for ticker column and tuple columns
    
    Args:
        df: Stock price dataframe
        price_col: Column containing price data
        periods: List of periods for calculating returns
        ticker_col: Column with ticker/symbol information (if any)
    
    Returns:
        Dataframe with added return features
    """
    result = df.copy()
    
    # Simple returns
    for period in periods:
        result[f'return_{period}d'] = result[price_col].pct_change(period)
    
    # Log returns (useful for mathematical operations)
    result['log_return_1d'] = np.log(result[price_col] / result[price_col].shift(1))
    
    # Cumulative returns - handle cases with and without ticker column
    # Without using groupby with tuple columns which causes errors
    try:
        # Add a year column for grouping
        result['_year'] = result.index.year
        
        # Calculate cumulative returns for each year
        # First, identify year boundaries
        year_starts = result['_year'] != result['_year'].shift(1)
        # Create a cumulative return that resets at the start of each year
        result['_cum_return_base'] = result[price_col] / result[price_col].shift(1)
        result.loc[year_starts, '_cum_return_base'] = 1.0
        # Calculate cumulative product
        result['cum_return_ytd'] = result['_cum_return_base'].cumprod() - 1
        
        # Clean up temporary columns
        result = result.drop(columns=['_year', '_cum_return_base'])
    except Exception as e:
        print(f"Warning: Error calculating cumulative returns: {e}")
        result['cum_return_ytd'] = np.nan
    
    # Rolling return metrics
    result['rolling_mean_20d'] = result['return_1d'].rolling(window=20).mean()
    result['rolling_std_20d'] = result['return_1d'].rolling(window=20).std()
    
    # Risk-adjusted return (Sharpe ratio proxy - without risk-free rate)
    result['rolling_sharpe_20d'] = result['rolling_mean_20d'] / result['rolling_std_20d']
    
    return result

# Use the new function instead
stock_features = calculate_return_features_with_ticker(
    stock_features, 
    price_col=price_cols['close'],
    ticker_col=ticker_col
)

# Define an updated volatility features function that accepts all needed parameters
def calculate_volatility_features_enhanced(df, return_col='return_1d', windows=[20, 60, 120], high_col=None, low_col=None, close_col=None):
    """
    Calculate volatility-based features with enhanced parameter support
    
    Args:
        df: Stock price dataframe
        return_col: Column containing return data
        windows: List of windows for rolling calculations
        high_col: Column containing high price data (optional)
        low_col: Column containing low price data (optional)
        close_col: Column containing close price data (optional)
    
    Returns:
        Dataframe with added volatility features
    """
    result = df.copy()
    
    # Check if the return column exists
    if return_col not in result.columns:
        print(f"Warning: '{return_col}' column not found. Creating it from close prices.")
        if close_col and close_col in result.columns:
            result[return_col] = result[close_col].pct_change()
        else:
            # Use the first numeric column if no specific column is available
            numeric_cols = result.select_dtypes(include=['number']).columns
            if len(numeric_cols) > 0:
                print(f"Using '{numeric_cols[0]}' to calculate returns.")
                result[return_col] = result[numeric_cols[0]].pct_change()
            else:
                print("Error: No numeric columns available for return calculation.")
                return result
    
    # Historical volatility (annualized)
    for window in windows:
        result[f'volatility_{window}d'] = result[return_col].rolling(window=window).std() * np.sqrt(252)
    
    # Calculate high-low range volatility if we have the necessary columns
    if high_col and low_col and close_col and all(col in result.columns for col in [high_col, low_col, close_col]):
        result['daily_range'] = (result[high_col] - result[low_col]) / result[close_col]
        result['rolling_range_20d'] = result['daily_range'].rolling(window=20).mean()
    
    return result

# Use the enhanced version
stock_features = calculate_volatility_features_enhanced(
    stock_features,
    return_col='return_1d',
    high_col=price_cols['high'],
    low_col=price_cols['low'],
    close_col=price_cols['close']
)

# Define an enhanced technical indicators function
def calculate_technical_indicators_enhanced(df, close_col='close', high_col='high', low_col='low', volume_col='volume'):
    """Calculate common technical indicators with enhanced parameter support"""
    result = df.copy()
    
    # Moving Averages
    for window in [20, 50, 200]:
        result[f'ma_{window}d'] = result[close_col].rolling(window=window).mean()
    
    # MACD components
    result['ema_12d'] = result[close_col].ewm(span=12, adjust=False).mean()
    result['ema_26d'] = result[close_col].ewm(span=26, adjust=False).mean()
    result['macd'] = result['ema_12d'] - result['ema_26d']
    result['macd_signal'] = result['macd'].ewm(span=9, adjust=False).mean()
    result['macd_hist'] = result['macd'] - result['macd_signal']
    
    # Relative Strength Index (RSI)
    delta = result[close_col].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    
    # Avoid division by zero
    rs = gain / loss.replace(0, np.nan)
    result['rsi_14d'] = 100 - (100 / (1 + rs))
    
    # Bollinger Bands
    result['bb_middle_20d'] = result[close_col].rolling(window=20).mean()
    result['bb_std_20d'] = result[close_col].rolling(window=20).std()
    result['bb_upper_20d'] = result['bb_middle_20d'] + 2 * result['bb_std_20d']
    result['bb_lower_20d'] = result['bb_middle_20d'] - 2 * result['bb_std_20d']
    
    # Relative volume if volume column is available
    if volume_col in result.columns:
        result['volume_ratio_20d'] = result[volume_col] / result[volume_col].rolling(window=20).mean()
    
    return result

# Use the enhanced version
stock_features = calculate_technical_indicators_enhanced(
    stock_features, 
    close_col=price_cols['close'],
    high_col=price_cols['high'],
    low_col=price_cols['low'],
    volume_col=price_cols['volume']
)

# Define an enhanced momentum features function
def calculate_momentum_features_enhanced(df, close_col='close', periods=[20, 60, 120, 252]):
    """Calculate momentum-based features with enhanced parameter support"""
    result = df.copy()
    
    # Momentum (price change over period)
    for period in periods:
        result[f'momentum_{period}d'] = result[close_col].diff(period)
    
    # Rate of Change (percent price change over period)
    for period in periods:
        result[f'roc_{period}d'] = result[close_col].pct_change(period) * 100
    
    # Recent performance percentile (where current price ranks in its recent history)
    for period in periods:
        result[f'price_percentile_{period}d'] = result[close_col].rolling(period).apply(
            lambda x: pd.Series(x).rank(pct=True).iloc[-1])
    
    return result

# Use the enhanced version  
stock_features = calculate_momentum_features_enhanced(stock_features, close_col=price_cols['close'])
stock_features = calculate_drawdown_features(stock_features, return_col='return_1d')

# Process economic indicators
print("Engineering features for economic indicators...")
economic_features = calculate_economic_indicators_features(economic_indicators)
economic_features = calculate_yield_curve_features(economic_features)

# Define an enhanced version of the market regime features function
def engineer_market_regime_features_enhanced(stock_data, economic_data, fama_french_data, ticker_col=None):
    """Create features for market regime classification with enhanced flexibility for MultiIndex dataframes"""
    # Make a copy to avoid modifying the original
    stock_data_copy = stock_data.copy()
    
    # Print column information
    print("\nAnalyzing stock data structure:")
    print(f"Column index levels: {stock_data_copy.columns.nlevels}")
    print(f"Index type: {type(stock_data_copy.columns)}")
    
    # Handle MultiIndex columns
    if isinstance(stock_data_copy.columns, pd.MultiIndex):
        print("Detected MultiIndex columns. Restructuring data...")
        
        # Identify features (columns with empty second level)
        feature_cols = [col for col in stock_data_copy.columns if col[1] == '']
        price_cols = [col for col in stock_data_copy.columns if col[1] != '']
        
        print(f"Found {len(feature_cols)} feature columns and {len(price_cols)} price columns")
        
        # Select only SPY (or another main index) close price for market representation
        market_tickers = ['SPY', 'IVV', '^GSPC', '^SPX']  # Different representations of S&P 500
        market_ticker = None
        
        for ticker in market_tickers:
            market_close_candidates = [(t, c) for t, c in price_cols if t == ticker and 'close' in c.lower()]
            if market_close_candidates:
                market_ticker = ticker
                market_close = market_close_candidates[0]
                print(f"Using {market_close} as market price")
                break
        
        if not market_ticker:
            # If no S&P 500 representation, use the first ticker's close price
            first_ticker = price_cols[0][0]
            market_close_candidates = [(t, c) for t, c in price_cols if t == first_ticker and 'close' in c.lower()]
            if market_close_candidates:
                market_close = market_close_candidates[0]
                print(f"Using {market_close} as market price (fallback)")
            else:
                # Last resort, use the first price column
                market_close = price_cols[0]
                print(f"Using {market_close} as market price (last resort)")
        
        # Create a simplified DataFrame with just market data and features
        # First, let's extract just the needed feature columns
        market_features = pd.DataFrame(index=stock_data_copy.index)
        
        # Add market return if it exists
        if ('return_1d', '') in stock_data_copy.columns:
            market_features['market_return'] = stock_data_copy[('return_1d', '')]
        else:
            # Calculate market return from price
            market_features['market_return'] = stock_data_copy[market_close].pct_change()
        
        # Add market volatility if it exists
        if ('volatility_20d', '') in stock_data_copy.columns:
            market_features['market_volatility'] = stock_data_copy[('volatility_20d', '')]
        else:
            # Calculate market volatility from return
            market_features['market_volatility'] = market_features['market_return'].rolling(window=20).std() * np.sqrt(252)
        
        # Add other useful features if they exist
        if ('drawdown', '') in stock_data_copy.columns:
            market_features['market_drawdown'] = stock_data_copy[('drawdown', '')]
        
        if ('rsi_14d', '') in stock_data_copy.columns:
            market_features['market_rsi'] = stock_data_copy[('rsi_14d', '')]
        
        if ('volume_ratio_20d', '') in stock_data_copy.columns:
            market_features['market_volume_ratio'] = stock_data_copy[('volume_ratio_20d', '')]
        
        # Use this simplified DataFrame for further calculations
        market_data = market_features
    else:
        # For non-MultiIndex, use the same approach as before
        print("Using standard column structure")
        
        # Define aggregation columns
        aggregation_cols = {}
        if 'return_1d' in stock_data_copy.columns:
            aggregation_cols['return_1d'] = 'mean'
        if 'volatility_20d' in stock_data_copy.columns:
            aggregation_cols['volatility_20d'] = 'mean'
        if 'volume_ratio_20d' in stock_data_copy.columns:
            aggregation_cols['volume_ratio_20d'] = 'mean'
        if 'drawdown' in stock_data_copy.columns:
            aggregation_cols['drawdown'] = 'mean'
        
        # Create market data
        if aggregation_cols:
            if ticker_col and ticker_col in stock_data_copy.columns:
                try:
                    market_data = stock_data_copy.groupby(stock_data_copy.index).agg(aggregation_cols)
                except Exception as e:
                    print(f"Error during aggregation: {e}")
                    market_data = stock_data_copy.copy()
            else:
                market_data = stock_data_copy
        else:
            market_data = stock_data_copy
    
    # Add rolling statistics 
    if 'market_return' in market_data.columns:
        try:
            market_data['rolling_return_20d'] = market_data['market_return'].rolling(window=20).mean() * 20  # Approx monthly return
            market_data['rolling_return_60d'] = market_data['market_return'].rolling(window=60).mean() * 60  # Approx quarterly return
            
            # Trend regime features
            market_data['market_ma_20d'] = market_data['market_return'].rolling(window=20).mean()
            ma_long_window = min(100, max(30, len(market_data) // 3))
            market_data['market_ma_long'] = market_data['market_return'].rolling(window=ma_long_window).mean()
            market_data['trend_indicator'] = np.where(
                market_data['market_ma_20d'] > market_data['market_ma_long'], 
                1, 
                -1
            )
        except Exception as e:
            print(f"Error calculating return metrics: {e}")
    
    # Join with economic indicators if available
    if economic_data is not None and not economic_data.empty:
        try:
            # Check if economic data has a different index level
            if (isinstance(economic_data.index, pd.MultiIndex) and 
                not isinstance(market_data.index, pd.MultiIndex)):
                # Simplify economic_data index
                economic_data_simplified = economic_data.copy()
                if isinstance(economic_data_simplified.index, pd.MultiIndex):
                    economic_data_simplified.index = economic_data_simplified.index.get_level_values(0)
                regime_features = market_data.join(economic_data_simplified, how='left')
            elif (not isinstance(economic_data.index, pd.MultiIndex) and 
                  isinstance(market_data.index, pd.MultiIndex)):
                # Simplify market_data index
                market_data_simplified = market_data.copy()
                if isinstance(market_data_simplified.index, pd.MultiIndex):
                    market_data_simplified.index = market_data_simplified.index.get_level_values(0)
                regime_features = market_data_simplified.join(economic_data, how='left')
            else:
                # Both have same index structure
                regime_features = market_data.join(economic_data, how='left')
        except Exception as e:
            print(f"Error joining economic data: {e}")
            print("Using market data without economic indicators")
            regime_features = market_data
    else:
        regime_features = market_data
    
    # Basic market regime classification
    if 'market_volatility' in regime_features.columns and 'market_return' in regime_features.columns:
        try:
            # Use 75th percentile for volatility threshold
            vol_threshold = regime_features['market_volatility'].quantile(0.75)
            
            conditions = [
                (regime_features['market_volatility'] > vol_threshold) & 
                (regime_features['market_return'] < 0),
                
                (regime_features['market_volatility'] <= vol_threshold) & 
                (regime_features['market_return'] < 0),
                
                (regime_features['market_volatility'] > vol_threshold) & 
                (regime_features['market_return'] >= 0),
                
                (regime_features['market_volatility'] <= vol_threshold) & 
                (regime_features['market_return'] >= 0)
            ]
            
            choices = ['bear_volatile', 'bear_stable', 'bull_volatile', 'bull_stable']
            regime_features['market_regime_basic'] = np.select(conditions, choices, default='undefined')
        except Exception as e:
            print(f"Error creating market regime classification: {e}")
    
    print(f"Created market regime features DataFrame with shape: {regime_features.shape}")
    print(f"Columns: {regime_features.columns.tolist()[:10]}...")
    
    return regime_features
    
# Use the enhanced version
market_regime_features = engineer_market_regime_features_enhanced(
    stock_features, economic_features, fama_french_factors, ticker_col=ticker_col
)

Engineering features for stock data...
Using '('GLD', 'Stock Splits')' as the ticker/symbol identifier column.
Using '('GLD', 'Close')' as the close price column.
Identified columns: {'close': ('GLD', 'Close'), 'high': ('GLD', 'High'), 'low': ('GLD', 'Low'), 'volume': ('GLD', 'Volume')}
Engineering features for economic indicators...

Analyzing stock data structure:
Column index levels: 2
Index type: <class 'pandas.core.indexes.multi.MultiIndex'>
Detected MultiIndex columns. Restructuring data...
Found 46 feature columns and 48 price columns
Using ('SPY', 'Close') as market price
Created market regime features DataFrame with shape: (3812, 41)
Columns: ['market_return', 'market_volatility', 'market_drawdown', 'market_rsi', 'market_volume_ratio', 'rolling_return_20d', 'rolling_return_60d', 'market_ma_20d', 'market_ma_long', 'trend_indicator']...


## 8. Feature Selection and Analysis

### 8.1 Correlation Analysis

In [14]:
def analyze_feature_correlations(df, target_col=None, threshold=0.7):
    """Analyze correlations between features"""
    # Calculate correlation matrix
    corr_matrix = df.select_dtypes(include=['number']).corr()
    
    # Plot the correlation heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', center=0)
    plt.title('Feature Correlation Heatmap')
    plt.tight_layout()
    plt.show()
    
    # Find highly correlated feature pairs
    high_corr_features = []
    
    # Get the upper triangle of the correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    
    # Find features with correlation greater than threshold
    for col in upper.columns:
        high_corr = upper[col][abs(upper[col]) > threshold].index.tolist()
        for feature in high_corr:
            high_corr_features.append((col, feature, upper[col][feature]))
    
    # Sort by correlation value
    high_corr_features.sort(key=lambda x: abs(x[2]), reverse=True)
    
    print(f"Highly correlated feature pairs (|r| > {threshold}):")
    for feat1, feat2, corr in high_corr_features:
        print(f"{feat1} - {feat2}: {corr:.4f}")
    
    # If target column is specified, show correlation with target
    if target_col and target_col in corr_matrix.columns:
        target_corr = corr_matrix[target_col].sort_values(ascending=False)
        print(f"\nFeature correlations with {target_col}:")
        print(target_corr)
    
    return high_corr_features

### 8.2 Feature Importance Analysis

In [15]:
def analyze_feature_importance(df, target_col, top_n=20):
    """Analyze feature importance using a simple model"""
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.preprocessing import StandardScaler
    
    # Prepare data
    X = df.select_dtypes(include=['number']).drop(columns=[target_col], errors='ignore')
    y = df[target_col]
    
    # Handle missing values for this analysis
    X = X.fillna(X.mean())
    X = StandardScaler().fit_transform(X)
    
    # Train a simple random forest model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X, y)
    
    # Get feature importance
    feature_importance = pd.DataFrame({
        'Feature': df.select_dtypes(include=['number']).drop(columns=[target_col], errors='ignore').columns,
        'Importance': model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    # Plot top features
    plt.figure(figsize=(10, 8))
    sns.barplot(x='Importance', y='Feature', data=feature_importance.head(top_n))
    plt.title(f'Top {top_n} Features by Importance for {target_col}')
    plt.tight_layout()
    plt.show()
    
    return feature_importance

## 9. Save Processed Features

In [16]:
# Save the engineered features
def save_features(dfs, filenames):
    """Save processed dataframes to parquet files"""
    for df, filename in zip(dfs, filenames):
        filepath = PROCESSED_DATA_PATH / filename
        df.to_parquet(filepath)
        print(f"Saved {filename} - Shape: {df.shape}")

# Save all the feature sets
feature_dfs = [
    stock_features,
    economic_features,
    market_regime_features
]

feature_filenames = [
    "stock_features.parquet",
    "economic_features.parquet",
    "market_regime_features.parquet"
]

save_features(feature_dfs, feature_filenames)

Saved stock_features.parquet - Shape: (3812, 94)
Saved economic_features.parquet - Shape: (4005, 30)
Saved market_regime_features.parquet - Shape: (3812, 41)


## 10. Summary and Next Steps

In [17]:
print("Feature Engineering Summary:")
print(f"- Stock features: {stock_features.shape[1]} features generated")
print(f"- Economic features: {economic_features.shape[1]} features generated")
print(f"- Market regime features: {market_regime_features.shape[1]} features generated")

print("\nNext Steps:")
print("1. Evaluate the usefulness of engineered features for portfolio construction")
print("2. Refine market regime classification using unsupervised learning")
print("3. Implement portfolio construction models using the engineered features")
print("4. Backtest different allocation strategies based on these features")

Feature Engineering Summary:
- Stock features: 94 features generated
- Economic features: 30 features generated
- Market regime features: 41 features generated

Next Steps:
1. Evaluate the usefulness of engineered features for portfolio construction
2. Refine market regime classification using unsupervised learning
3. Implement portfolio construction models using the engineered features
4. Backtest different allocation strategies based on these features


## 11. Verify Saved Engineered Features

In [18]:
# Verify the saved files
processed_files = os.listdir(PROCESSED_DATA_PATH)
print("Files in processed data directory:")
for file in processed_files:
    filepath = PROCESSED_DATA_PATH / file
    # Load the file to check it can be read back
    df = pd.read_parquet(filepath)
    print(f"{file} - Shape: {df.shape} - Columns: {len(df.columns)} - Index type: {type(df.index)}")

Files in processed data directory:
economic_features.parquet - Shape: (4005, 30) - Columns: 30 - Index type: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
market_regime_features.parquet - Shape: (3812, 41) - Columns: 41 - Index type: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
stock_features.parquet - Shape: (3812, 94) - Columns: 94 - Index type: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>


## 12. Document Key Insights

In [21]:
# Analyze processed data to gather key insights
print("Gathering key insights from processed data...")

# Load the processed data files
stock_features = pd.read_parquet(PROCESSED_DATA_PATH / "stock_features.parquet")
economic_features = pd.read_parquet(PROCESSED_DATA_PATH / "economic_features.parquet")
market_regime_features = pd.read_parquet(PROCESSED_DATA_PATH / "market_regime_features.parquet")

Gathering key insights from processed data...


### 12.1 Time Period and Frequency Analysis

In [20]:
print("\n1. TIME PERIOD AND FREQUENCY ANALYSIS")
# Ensure index is datetime
for df_name, df in zip(['Stock features', 'Economic features', 'Market regime features'], 
                       [stock_features, economic_features, market_regime_features]):
    if not pd.api.types.is_datetime64_dtype(df.index):
        try:
            df.index = pd.to_datetime(df.index)
        except:
            print(f"Could not convert index to datetime for {df_name}")
            continue
    
    start_date = df.index.min()
    end_date = df.index.max()
    total_days = (end_date - start_date).days
    avg_days_between = total_days / (len(df) - 1) if len(df) > 1 else 0
    
    print(f"{df_name}:")
    print(f"  - Date range: {start_date.date()} to {end_date.date()} ({total_days} days)")
    print(f"  - Number of observations: {len(df)}")
    print(f"  - Average days between observations: {avg_days_between:.2f}")
    
    if avg_days_between < 2:
        freq = "daily"
    elif avg_days_between < 8:
        freq = "weekly"
    elif avg_days_between < 32:
        freq = "monthly"
    elif avg_days_between < 95:
        freq = "quarterly"
    else:
        freq = "annual or irregular"
    
    print(f"  - Approximate frequency: {freq}")


1. TIME PERIOD AND FREQUENCY ANALYSIS
Stock features:
  - Date range: 2010-01-04 to 2025-02-27 (5533 days)
  - Number of observations: 3812
  - Average days between observations: 1.45
  - Approximate frequency: daily
Economic features:
  - Date range: 2010-01-01 to 2025-02-27 (5536 days)
  - Number of observations: 4005
  - Average days between observations: 1.38
  - Approximate frequency: daily
Market regime features:
  - Date range: 2010-01-04 to 2025-02-27 (5533 days)
  - Number of observations: 3812
  - Average days between observations: 1.45
  - Approximate frequency: daily


### 12.2 Economic Indicator Analysis

In [22]:
print("\n2. ECONOMIC INDICATORS ANALYSIS")
if len(economic_features.columns) > 0:
    print(f"Number of economic indicators: {len(economic_features.columns)}")
    print("\nSample of economic indicators:")
    print(economic_features.columns.tolist()[:10])
    
    # Calculate basic statistics for economic indicators
    print("\nSummary statistics for economic indicators:")
    display(economic_features.describe().T.sort_values('mean', ascending=False).head(10))
else:
    print("No economic indicators found in the dataset.")


2. ECONOMIC INDICATORS ANALYSIS
Number of economic indicators: 30

Sample of economic indicators:
['GDP', 'UNRATE', 'CPIAUCSL', 'FEDFUNDS', 'T10Y2Y', 'GDP_pct_change', 'GDP_zscore', 'GDP_ma_3m', 'GDP_ma_12m', 'GDP_trend']

Summary statistics for economic indicators:


,count,mean,std,min,25%,50%,75%,max
GDP,60.0,20549.782617,4310.087947,14764.610000,17132.473750,19565.619000,22834.810000,29719.647000
CPIAUCSL,181.0,254.491083,28.788096,217.199000,233.669000,244.243000,266.625000,319.086000
UNRATE,181.0,5.788398,2.234609,3.400000,3.900000,5.000000,7.500000,14.800000
FEDFUNDS,181.0,1.246409,1.728006,0.050000,0.090000,0.190000,1.830000,5.330000
T10Y2Y_zscore,1.0,1.085354,NaN,1.085354,1.085354,1.085354,1.085354,1.085354
T10Y2Y_ma_12m,1870.0,1.022017,0.979190,-0.939167,0.240208,1.013333,1.721667,2.858333
T10Y2Y_ma_3m,3386.0,1.016017,0.971170,-1.026667,0.236667,1.023333,1.719167,2.900000
T10Y2Y,3791.0,1.012933,0.970092,-1.080000,0.240000,1.010000,1.710000,2.910000
FEDFUNDS_pct_change,4004.0,0.002142,0.049717,-0.923077,0.000000,0.000000,0.000000,1.500000
GDP_pct_change,4004.0,0.000178,0.002551,-0.082485,0.000000,0.000000,0.000000,0.087739


### 12.3 Asset Class Performance Analysis

In [23]:
print("\n3. ASSET CLASS PERFORMANCE ANALYSIS")
# Identify ETF tickers in the stock data
if isinstance(stock_features.columns, pd.MultiIndex):
    # For MultiIndex columns
    tickers = sorted(set([col[0] for col in stock_features.columns if isinstance(col, tuple)]))
    
    # Extract close prices for each ticker
    close_prices = {}
    returns = {}
    
    for ticker in tickers:
        # Find close price column for this ticker
        close_cols = [col for col in stock_features.columns if isinstance(col, tuple) and col[0] == ticker and 'close' in col[1].lower()]
        if close_cols:
            close_col = close_cols[0]
            close_prices[ticker] = stock_features[close_col]
            # Calculate returns
            returns[ticker] = stock_features[close_col].pct_change()
    
    # Create DataFrame of returns
    returns_df = pd.DataFrame(returns)
    
    # Calculate annualized metrics
    annualized_metrics = pd.DataFrame(index=tickers)
    annualized_metrics['Ann_Return'] = returns_df.mean() * 252 * 100  # Assuming daily data
    annualized_metrics['Ann_Volatility'] = returns_df.std() * np.sqrt(252) * 100
    annualized_metrics['Sharpe_Ratio'] = annualized_metrics['Ann_Return'] / annualized_metrics['Ann_Volatility']
    
    # Calculate max drawdown
    cum_returns = (1 + returns_df).cumprod()
    running_max = cum_returns.cummax()
    drawdown = (cum_returns / running_max - 1)
    annualized_metrics['Max_Drawdown'] = drawdown.min() * 100
    
    # Display results sorted by return
    print("Annualized performance metrics by asset (%):")
    display(annualized_metrics.sort_values('Ann_Return', ascending=False))
    
    # Calculate correlation matrix
    print("\nAsset correlation matrix:")
    display(returns_df.corr())
else:
    print("Cannot extract ticker performance from non-MultiIndex columns.")



3. ASSET CLASS PERFORMANCE ANALYSIS
Annualized performance metrics by asset (%):


,Ann_Return,Ann_Volatility,Sharpe_Ratio,Max_Drawdown
SPY,14.132227,17.014553,0.830596,-33.717264
VNQ,10.918854,20.674756,0.528125,-42.398173
VEA,7.186258,18.488581,0.388686,-35.735079
GLD,7.026526,15.482892,0.453825,-45.555013
VWO,5.319318,20.683012,0.257183,-36.392357
AGG,2.517139,4.749973,0.529927,-18.432950
bb_lower_20d,NaN,NaN,NaN,NaN
bb_middle_20d,NaN,NaN,NaN,NaN
bb_std_20d,NaN,NaN,NaN,NaN
bb_upper_20d,NaN,NaN,NaN,NaN



Asset correlation matrix:


,AGG,GLD,SPY,VEA,VNQ,VWO
AGG,1.000000,0.301862,0.003325,0.027621,0.173118,0.019880
GLD,0.301862,1.000000,0.056152,0.142704,0.116384,0.170011
SPY,0.003325,0.056152,1.000000,0.870928,0.755611,0.779166
VEA,0.027621,0.142704,0.870928,1.000000,0.696940,0.855854
VNQ,0.173118,0.116384,0.755611,0.696940,1.000000,0.607542
VWO,0.019880,0.170011,0.779166,0.855854,0.607542,1.000000


### 12.4 Feature Correlation Analysis

In [24]:
print("\n4. FEATURE CORRELATION ANALYSIS")
# Try to find return columns for correlation analysis
return_cols = []

if isinstance(stock_features.columns, pd.MultiIndex):
    # For MultiIndex columns, look for return_1d in the first level
    return_cols = [col for col in stock_features.columns if col[0] == 'return_1d' or 
                  (isinstance(col, tuple) and len(col) >= 2 and col[1] == '' and col[0] == 'return_1d')]
else:
    # For regular columns
    return_cols = [col for col in stock_features.columns if 'return_1d' in str(col)]

# If we found return columns, analyze correlations
if return_cols:
    return_col = return_cols[0]
    
    # Create a DataFrame for correlation analysis
    if isinstance(stock_features.columns, pd.MultiIndex):
        # For MultiIndex, select columns with empty second level (features)
        feature_cols = [col for col in stock_features.columns if isinstance(col, tuple) and col[1] == '']
        correlation_df = stock_features[feature_cols]
    else:
        correlation_df = stock_features
    
    # Calculate correlations with returns
    if return_col in correlation_df.columns:
        correlations = correlation_df.corr()[return_col].sort_values(ascending=False)
        
        print("Top features correlated with returns:")
        display(correlations.head(10))
        
        print("\nBottom features correlated with returns:")
        display(correlations.tail(10))
    else:
        print(f"Return column {return_col} not found in filtered DataFrame.")
else:
    print("No return columns found for correlation analysis.")



4. FEATURE CORRELATION ANALYSIS
Top features correlated with returns:


Ticker                Price
return_1d                      1.000000
log_return_1d                  0.999923
return_5d                      0.446694
price_percentile_20d           0.391352
rsi_14d                        0.252408
price_percentile_60d           0.238417
return_20d                     0.209229
roc_20d                        0.209229
rolling_mean_20d               0.209090
rolling_sharpe_20d             0.206996
Name: (return_1d, ), dtype: float64


Bottom features correlated with returns:


Ticker            Price
ma_20d                     0.001424
bb_lower_20d              -0.000765
macd_signal               -0.004352
running_max               -0.007333
volatility_120d           -0.008760
volatility_20d            -0.014341
rolling_std_20d           -0.014341
volatility_60d            -0.016412
volume_ratio_20d          -0.084820
daily_range               -0.102490
Name: (return_1d, ), dtype: float64

### 12.5 Market Regime Analysis

In [25]:
print("\n5. MARKET REGIME ANALYSIS")
if 'market_regime_basic' in market_regime_features.columns:
    regime_counts = market_regime_features['market_regime_basic'].value_counts()
    regime_pct = market_regime_features['market_regime_basic'].value_counts(normalize=True) * 100
    
    print("Market regime distribution:")
    for regime, count in regime_counts.items():
        print(f"  - {regime}: {count} days ({regime_pct[regime]:.2f}%)")
    
    # Analyze returns by regime
    if 'market_return' in market_regime_features.columns:
        print("\nReturn characteristics by market regime:")
        regime_returns = market_regime_features.groupby('market_regime_basic')['market_return'].agg(
            ['mean', 'std', 'min', 'max', 'count']
        )
        regime_returns['mean'] *= 100  # Convert to percentage
        regime_returns['std'] *= 100
        regime_returns['min'] *= 100
        regime_returns['max'] *= 100
        regime_returns['annualized_return'] = regime_returns['mean'] * 252
        regime_returns['annualized_vol'] = regime_returns['std'] * np.sqrt(252)
        regime_returns['sharpe'] = regime_returns['annualized_return'] / regime_returns['annualized_vol']
        
        display(regime_returns.sort_values('sharpe', ascending=False))
else:
    print("No market regime classification found.")


5. MARKET REGIME ANALYSIS
Market regime distribution:
  - bull_stable: 1512 days (39.66%)
  - bear_stable: 1332 days (34.94%)
  - bull_volatile: 494 days (12.96%)
  - bear_volatile: 454 days (11.91%)
  - undefined: 20 days (0.52%)

Return characteristics by market regime:


,mean,std,min,max,count,annualized_return,annualized_vol,sharpe
market_regime_basic,,,,,,,,
bull_volatile,1.008120,0.849978,0.000000,4.903838,494,254.046208,13.492988,18.828017
bull_stable,0.588090,0.505759,0.000000,3.480887,1512,148.198609,8.028679,18.458654
undefined,-0.063154,1.196361,-2.313481,2.255568,19,-15.914787,18.991639,-0.837989
bear_volatile,-1.076287,1.028147,-8.780826,-0.004643,454,-271.224420,16.321322,-16.617798
bear_stable,-0.593924,0.519476,-3.471113,-0.005999,1332,-149.668743,8.246428,-18.149525


### Data Quality Analysis

In [26]:
print("\n6. DATA QUALITY ANALYSIS")
for df_name, df in zip(['Stock features', 'Economic features', 'Market regime features'], 
                       [stock_features, economic_features, market_regime_features]):
    missing_count = df.isna().sum().sum()
    total_cells = df.shape[0] * df.shape[1]
    missing_pct = missing_count / total_cells * 100 if total_cells > 0 else 0
    
    print(f"{df_name}:")
    print(f"  - Total observations: {df.shape[0]}, Total features: {df.shape[1]}")
    print(f"  - Missing values: {missing_count} ({missing_pct:.2f}%)")
    
    # Check for columns with high missing values
    if missing_count > 0:
        col_missing = df.isna().sum().sort_values(ascending=False)
        high_missing = col_missing[col_missing > 0.1 * df.shape[0]]  # >10% missing
        if len(high_missing) > 0:
            print(f"  - Columns with >10% missing values: {len(high_missing)}")
            print("    Top 5:", high_missing.index.tolist()[:5])


6. DATA QUALITY ANALYSIS
Stock features:
  - Total observations: 3812, Total features: 94
  - Missing values: 2528 (0.71%)
Economic features:
  - Total observations: 4005, Total features: 30
  - Missing values: 88616 (73.75%)
  - Columns with >10% missing values: 24
    Top 5: ['UNRATE_ma_3m', 'UNRATE_zscore', 'GDP_trend', 'GDP_ma_12m', 'GDP_ma_3m']
Market regime features:
  - Total observations: 3812, Total features: 41
  - Missing values: 84272 (53.92%)
  - Columns with >10% missing values: 24
    Top 5: ['FEDFUNDS_ma_12m', 'GDP_ma_3m', 'GDP_ma_12m', 'GDP_trend', 'UNRATE_zscore']


## Key Insights from Feature Engineering
### Time Period and Frequency

The dataset covers a long period from January 2010 to February 2025, spanning over 15 years and including approximately 3,800-4,000 daily observations
The daily frequency provides granular visibility into market movements across multiple economic cycles
The time range captures several significant market events including the post-2008 recovery, bull market of the 2010s, COVID-19 crash, and subsequent recovery

## Asset Class Performance Insights

- US equities (SPY) delivered the strongest returns with annualized return of 14.13% and a Sharpe ratio of 0.83
- Real Estate (VNQ) provided the second-best returns (10.92%) but with higher volatility (20.67%)
- Fixed income (AGG) showed the lowest volatility (4.75%) and modest returns (2.52%), consistent with its risk profile
- Gold (GLD) offered moderate returns (7.03%) with reasonable volatility (15.48%)
- International equities (VEA, VWO) underperformed US equities, with developed markets (VEA: 7.19%) outperforming emerging markets (VWO: 5.32%)
- The maximum drawdowns varied significantly across asset classes, from -18.43% for bonds (AGG) to -45.56% for gold (GLD)

## Correlation Analysis

- US stocks (SPY) show strong correlations with international equities (VEA: 0.87, VWO: 0.78) and real estate (VNQ: 0.76)
- Gold (GLD) exhibits low correlation with equities (SPY: 0.06) and moderate correlation with bonds (AGG: 0.30), confirming its diversification benefits
- Bonds (AGG) demonstrate very low correlation with equities (SPY: 0.003), highlighting their effectiveness as a diversifier
- The correlation structure supports the traditional diversification principles across these major asset classes

## Feature Significance

- Short-term return features (return_1d, log_return_1d, return_5d) show the strongest correlation with future returns
- Price percentile indicators (price_percentile_20d, price_percentile_60d) have meaningful correlations, suggesting momentum effects
- RSI (Relative Strength Index) appears to have predictive value with a correlation of 0.25
- Technical indicators like moving averages (ma_20d) show minimal correlation with returns, suggesting limited predictive power
- Volatility measures (volatility_20d, volatility_60d, volatility_120d) show slight negative correlations with returns

## Market Regime Characteristics

- The market spent the largest portion of time (39.66%) in bull_stable regimes, followed by bear_stable (34.94%)
- Volatile regimes were less common, with bull_volatile at 12.96% and bear_volatile at 11.91% of days
- Performance varied dramatically across regimes:

    - Bull_volatile regimes delivered the highest annualized returns (254.05%) with high volatility (13.49%)
    - Bull_stable periods provided strong returns (148.20%) with lower volatility (8.03%)
    - Bear regimes showed substantial negative returns, with bear_volatile (-271.22%) and bear_stable (-149.67%)
    - The highest Sharpe ratios occurred during bull regimes, especially bull_volatile (18.83)



## Economic Indicators Insights

- Key macroeconomic variables include GDP, inflation (CPIAUCSL), unemployment (UNRATE), interest rates (FEDFUNDS), and yield curve spreads (T10Y2Y)
- The unemployment rate varied significantly from 3.4% to 14.8%, capturing both strong economic periods and the COVID-19 pandemic impact
- The Federal Funds Rate ranged from near-zero (0.05%) to 5.33%, reflecting the full monetary policy cycle
- The dataset includes derived features like percentage changes, z-scores, moving averages, and trend indicators to capture different aspects of economic conditions

## Data Quality Considerations

- Stock features have high quality with minimal missing values (0.71%)
- Economic features contain significant missing data (73.75%), particularly in derived features like moving averages and z-scores
- Market regime features also show substantial missing values (53.92%), largely due to their dependence on economic indicators
- Key economic indicators with the most missing data include unemployment and GDP metrics, which likely reflects their less frequent release schedule compared to market data

## Engineering Challenges and Solutions

- The MultiIndex column structure required specialized approaches for feature engineering and analysis
- The integration of market data (daily) with economic data (often released monthly or quarterly) created alignment challenges
- The feature engineering process successfully created a comprehensive set of 94 stock features, 30 economic features, and 41 market regime features
- The approach clearly separated different market regimes, enabling regime-based investment strategies

These insights provide a solid foundation for developing baseline portfolio allocation models and more advanced strategies. The diverse asset classes, comprehensive feature set, and clearly defined market regimes offer multiple approaches for constructing and evaluating investment portfolios across different economic conditions.

## 13. Identify Potential Issues

In [19]:
# Check for missing values in each dataset
print("\nMissing value analysis:")
for df_name, df_file in zip(
    ['Stock features', 'Economic features', 'Market regime features'],
    ['stock_features.parquet', 'economic_features.parquet', 'market_regime_features.parquet']
):
    df = pd.read_parquet(PROCESSED_DATA_PATH / df_file)
    missing_count = df.isna().sum().sum()
    missing_pct = missing_count / (df.shape[0] * df.shape[1]) * 100
    print(f"{df_name}: {missing_count} missing values ({missing_pct:.2f}%)")
    
    # Show columns with the most missing values
    col_missing = df.isna().sum().sort_values(ascending=False)
    if col_missing.max() > 0:
        print("Top 5 columns with missing values:")
        print(col_missing[col_missing > 0].head())

# Check for data points at the boundaries (potential outliers)
print("\nOutlier analysis:")
for df_name, df_file in zip(
    ['Stock features', 'Economic features', 'Market regime features'],
    ['stock_features.parquet', 'economic_features.parquet', 'market_regime_features.parquet']
):
    df = pd.read_parquet(PROCESSED_DATA_PATH / df_file)
    
    # Select only numeric columns
    numeric_df = df.select_dtypes(include=['number'])
    
    # Calculate z-scores for each column
    z_scores = (numeric_df - numeric_df.mean()) / numeric_df.std()
    
    # Count potential outliers (|z| > 3)
    outlier_count = (abs(z_scores) > 3).sum().sum()
    outlier_pct = outlier_count / (numeric_df.shape[0] * numeric_df.shape[1]) * 100
    print(f"{df_name}: {outlier_count} potential outliers ({outlier_pct:.2f}%)")

# Check for time periods with sparse data
print("\nTime period analysis:")
for df_name, df_file in zip(
    ['Stock features', 'Economic features', 'Market regime features'],
    ['stock_features.parquet', 'economic_features.parquet', 'market_regime_features.parquet']
):
    df = pd.read_parquet(PROCESSED_DATA_PATH / df_file)
    
    # Ensure index is datetime
    if not pd.api.types.is_datetime64_dtype(df.index):
        try:
            df.index = pd.to_datetime(df.index)
        except:
            print(f"Could not convert index to datetime for {df_name}")
            continue
    
    # Group by year and count observations
    yearly_counts = df.groupby(df.index.year).size()
    print(f"{df_name} observations by year:")
    print(yearly_counts)


Missing value analysis:
Stock features: 2528 missing values (0.71%)
Top 5 columns with missing values:
Ticker                 Price
roc_252d                        252
return_252d                     252
momentum_252d                   252
price_percentile_252d           251
ma_200d                         199
dtype: int64
Economic features: 88616 missing values (73.75%)
Top 5 columns with missing values:
UNRATE_ma_3m     4005
UNRATE_zscore    4005
GDP_trend        4005
GDP_ma_12m       4005
GDP_ma_3m        4005
dtype: int64
Market regime features: 84272 missing values (53.92%)
Top 5 columns with missing values:
FEDFUNDS_ma_12m    3812
GDP_ma_3m          3812
GDP_ma_12m         3812
GDP_trend          3812
UNRATE_zscore      3812
dtype: int64

Outlier analysis:
Stock features: 2639 potential outliers (0.74%)
Economic features: 208 potential outliers (0.17%)
Market regime features: 429 potential outliers (0.28%)

Time period analysis:
Stock features observations by year:
Date
2010    